# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [101]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [102]:
pokemon = pd.read_csv('Pokemon.csv')
pokemon.head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [103]:
# your code here

# Observamos primero la matriz de correlación para los atributos que queremos examinar
cols = pokemon.columns[4:11]
pokemon[cols].corr()

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
Total,1.000000,0.618748,0.736211,0.612787,0.747250,0.717609,0.575943
HP,0.618748,1.000000,0.422386,0.239622,0.362380,0.378718,0.175952
Attack,0.736211,0.422386,1.000000,0.438687,0.396362,0.263990,0.381240
Defense,0.612787,0.239622,0.438687,1.000000,0.223549,0.510747,0.015227
Sp. Atk,0.747250,0.362380,0.396362,0.223549,1.000000,0.506121,0.473018
Sp. Def,0.717609,0.378718,0.263990,0.510747,0.506121,1.000000,0.259133
Speed,0.575943,0.175952,0.381240,0.015227,0.473018,0.259133,1.000000


In [104]:
# Parece que Total está muy correlacionado con el resto de atributos

min_corr = pokemon[cols].corr().iloc[0,:].min()    # Correlación minima el la fila TOTAL de la matriz corr()

pokemon[cols[1:]].corr() >= min_corr   
# Podemos ver que no existe correlación significativa entre los atributos que suponemos influyen en Total

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
HP,True,False,False,False,False,False
Attack,False,True,False,False,False,False
Defense,False,False,True,False,False,False
Sp. Atk,False,False,False,True,False,False
Sp. Def,False,False,False,False,True,False
Speed,False,False,False,False,False,True


In [105]:
# Vamos a hacer esta comprobación para todas los tipos de pokemon. para ello nos ayudaremos del chalange 1
types = pd.Series(list(set(pokemon['Type 1'].unique().tolist() + pokemon['Type 2'].unique().tolist())))
types.dropna(inplace=True)


In [106]:
# Voy a explorar como son las correlaciones de total con el resto de atributos, en función del tipo de pokemon
# Solo voy a examinar la columna type1

type_corr_by_total = dict()

for typ in types:    # recorro todos lo tipos únicos de pokemon
    for col in cols:
        df = pokemon[cols][(pokemon['Type 1'] == typ)].corr()       # creo el df de la matriz de correlación sólo para el tipo
        max_corr = df.iloc[0].sort_values(ascending=False)[1]       # obtengo el max ordenando la primera fila (total), y excluyo el 1
        max_index = df.iloc[0].sort_values(ascending=False)[1:].idxmax()
        min_corr = df.iloc[0].min()
        min_index = df.iloc[0].idxmin()        
        type_corr_by_total[typ] ={'max_index':max_index,     # Guardo los datos en un dict
                                 'max_corr':max_corr,
                                 'min_index':min_index,
                                 'min_corr':min_corr}
pd.DataFrame(type_corr_by_total).T        # Paso el dict a un df y lo chequeo

# Se puede ver que la correlación entre total y el resto de atributos. Sobre todo en la tipo 'Flying'
# Pero no siempre la correlación máxima es la misma en función del tipo
# por lo que podemos intuir, que también el tipo de pokemon puede tener correlación con el total

,max_index,max_corr,min_index,min_corr
Fire,Sp. Atk,0.818396,Speed,0.527186
Grass,Attack,0.813379,Speed,0.60309
Ice,HP,0.717148,Speed,0.509519
Fairy,Sp. Atk,0.940679,Sp. Def,0.68836
Fighting,Attack,0.867709,HP,0.453321
Poison,Attack,0.881785,Speed,0.62352
Rock,Sp. Atk,0.707987,Defense,0.405458
Ground,Sp. Atk,0.867559,Speed,0.454975
Dragon,Sp. Atk,0.908732,Sp. Def,0.713734
Psychic,Attack,0.817163,HP,0.456808


In [107]:
# Discutiendo el ejercicio con mis comapañeros, he visto que la respuesta era más sencialla que todo el rollo anterior.

# Para todos los casos. Total es la suma de todos los atributos

(pokemon.Total == pokemon[cols[1:]].sum(axis=1)).value_counts()

True    800
dtype: int64

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [109]:
# your code here
dummies_1 = pd.get_dummies(pokemon['Type 1'],dtype='int')
dummies_2 = pd.get_dummies(pokemon['Type 2'],dtype='int')
pokemon = pd.concat([pokemon,dummies_1+dummies_2],axis=1)
pokemon.shape

(800, 31)

In [126]:
pokemon.head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Combo Type,Bug,Bug-Electric,Bug-Fighting,Bug-Fire,Bug-Flying,Bug-Ghost,Bug-Grass,Bug-Ground,Bug-Poison,Bug-Rock,Bug-Steel,Bug-Water,Dark,Dark-Dragon,Dark-Fighting,Dark-Fire,Dark-Flying,Dark-Ghost,Dark-Ice,Dark-Psychic,Dark-Steel,Dragon,Dragon-Electric,Dragon-Fairy,Dragon-Fire,Dragon-Flying,Dragon-Ground,Dragon-Ice,Dragon-Psychic,Electric,Electric-Dragon,Electric-Fairy,Electric-Fire,Electric-Flying,Electric-Ghost,Electric-Grass,Electric-Ice,Electric-Normal,Electric-Steel,Electric-Water,Fairy,Fairy-Flying,Fighting,Fighting-Dark,Fighting-Flying,Fighting-Psychic,Fighting-Steel,Fire,Fire-Dragon,Fire-Fighting,Fire-Flying,Fire-Ground,Fire-Normal,Fire-Psychic,Fire-Rock,Fire-Steel,Fire-Water,Flying,Flying-Dragon,Ghost,Ghost-Dark,Ghost-Dragon,Ghost-Fire,Ghost-Flying,Ghost-Grass,Ghost-Poison,Grass,Grass-Dark,Grass-Dragon,Grass-Fairy,Grass-Fighting,Grass-Flying,Grass-Ground,Grass-Ice,Grass-Poison,Grass-Psychic,Grass-Steel,Ground,Ground-Dark,Ground-Dragon,Ground-Electric,Ground-Fire,Ground-Flying,Ground-Ghost,Ground-Psychic,Ground-Rock,Ground-Steel,Ice,Ice-Flying,Ice-Ghost,Ice-Ground,Ice-Psychic,Ice-Water,Normal,Normal-Fairy,Normal-Fighting,Normal-Flying,Normal-Grass,Normal-Ground,Normal-Psychic,Normal-Water,Poison,Poison-Bug,Poison-Dark,Poison-Dragon,Poison-Fighting,Poison-Flying,Poison-Ground,Poison-Water,Psychic,Psychic-Dark,Psychic-Fairy,Psychic-Fighting,Psychic-Fire,Psychic-Flying,Psychic-Ghost,Psychic-Grass,Rock,Rock-Bug,Rock-Dark,Rock-Dragon,Rock-Fairy,Rock-Fighting,Rock-Flying,Rock-Grass,Rock-Ground,Rock-Ice,Rock-Psychic,Rock-Steel,Rock-Water,Steel,Steel-Dragon,Steel-Fairy,Steel-Fighting,Steel-Flying,Steel-Ghost,Steel-Ground,Steel-Psychic,Steel-Rock,Water,Water-Dark,Water-Dragon,Water-Electric,Water-Fairy,Water-Fighting,Water-Flying,Water-Ghost,Water-Grass,Water-Ground,Water-Ice,Water-Poison,Water-Psychic,Water-Rock,Water-Steel
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,Grass-Poison,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,Grass-Poison,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,Grass-Poison,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,Grass-Poison,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,Fire,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False,Fire,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0

## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [78]:
# your code here
df_corr_type = pokemon[['Total']+types.tolist()].corr()
df_corr_type

,Total,Fire,Grass,Ice,Fairy,Fighting,Poison,Rock,Ground,Dragon,Psychic,Ghost,Normal,Steel,Water,Bug,Dark,Flying,Electric
Total,1.000000,0.078726,-0.052592,0.060248,-0.036698,0.077786,-0.090441,0.032731,0.015060,0.229705,0.124688,0.003641,-0.105331,0.109703,-0.021665,-0.145781,0.056154,0.059383,0.020971
Fire,0.078726,1.000000,-0.108248,-0.065851,-0.067651,0.051130,-0.085471,-0.064677,-0.022621,-0.038069,-0.061243,-0.013459,-0.085097,-0.056108,-0.114850,-0.060536,-0.020368,-0.028855,-0.057104
Grass,-0.052592,-0.108248,1.000000,-0.027476,-0.048756,-0.051171,0.110372,-0.072829,-0.097033,-0.078818,-0.094009,0.075316,-0.117156,-0.061537,-0.126894,-0.034430,-0.048339,-0.081366,-0.078818
Ice,0.060248,-0.065851,-0.027476,1.000000,-0.051232,-0.059483,-0.064726,-0.017110,-0.003872,0.015173,-0.042311,-0.029914,-0.085366,-0.057042,0.000242,-0.070229,-0.010163,-0.049498,-0.033382
Fairy,-0.036698,-0.067651,-0.048756,-0.051232,1.000000,-0.061108,-0.066495,0.002212,-0.069360,-0.035541,0.027227,-0.056665,-0.001720,0.013155,-0.067703,-0.072148,-0.059864,-0.052669,-0.035541
Fighting,0.077786,0.051130,-0.051171,-0.059483,-0.061108,1.000000,-0.039613,-0.055090,-0.080531,-0.068775,0.000596,-0.065792,-0.071687,-0.005161,-0.073778,-0.048645,-0.007791,-0.086119,-0.068775
Poison,-0.090441,-0.085471,0.110372,-0.064726,-0.066495,-0.039613,1.000000,-0.081036,-0.053878,-0.055525,-0.103195,0.008736,-0.110800,-0.074036,-0.073986,0.121210,-0.018227,-0.067950,-0.074838
Rock,0.032731,-0.064677,-0.072829,-0.017110,0.002212,-0.055090,-0.081036,1.000000,0.072085,-0.032360,-0.069031,-0.069057,-0.106877,0.049202,0.011447,-0.003706,-0.033493,-0.048221,-0.072188
Ground,0.015060,-0.022621,-0.097033,-0.003872,-0.069360,-0.080531,-0.053878,0.072085,1.000000,0.052430,-0.079079,-0.035908,-0.102044,-0.020771,-0.006844,-0.063544,-0.023480,-0.060578,-0.059420
Dragon,0.229705,-0.038069,-0.078818,0.015173,-0.035541,-0.068775,-0.055525,-0.032360,0.052430,1.000000,-0.026557,-0.019410,-0.098702,-0.044417,-0.083285,-0.081200,-0.003963,0.026237,-0.024000


In [99]:
# Obtengo la primera línea que es la correlación de Total, excluyo primer valor que es 1, que es la correlación Total-Total
max_corr = df_corr_type.iloc[0,1:].max()
id_max = df_corr_type.iloc[0,1:].idxmax()
f'La correlación máxima es: {round(max_corr,4)}, y es para el tipo {id_max}'

'La correlación máxima es: 0.2297, y es para el tipo Dragon'

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [132]:
# your code here

# La primera hipótesis que podríamos hacer es que la combincación Dragon - Psychic sería la optima,
#  ya que en el apartado anterior vimos que eran las dos que mayor correlación teníamos
df_corr_type.iloc[0,1:].sort_values(ascending=False).head(2)

Dragon     0.229705
Psychic    0.124688
Name: Total, dtype: float64

In [111]:
# Voy a comprobar si es así, y ver la correlación de todas las combinaciones con Total

# Me voy ayudar de la función creada en el chalange 1

# Reinicio el data_set, el get_dummies de antes ya no me es últil

pokemon = pd.read_csv('Pokemon.csv')

pokemon['Combo Type'] = ''
for index in pokemon.index:
    if pokemon.loc[index,'Type 2'] is np.nan:
        pokemon.loc[index,'Combo Type'] = pokemon.loc[index,'Type 1']
    else:
        pokemon.loc[index,'Combo Type'] = f"{pokemon.loc[index,'Type 1']}-{pokemon.loc[index,'Type 2']}"
        
dummies_combo = pd.get_dummies(pokemon['Combo Type'],dtype='int')
pokemon = pd.concat([pokemon,dummies_combo],axis=1)
pokemon.head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Combo Type,Bug,Bug-Electric,Bug-Fighting,Bug-Fire,Bug-Flying,Bug-Ghost,Bug-Grass,Bug-Ground,Bug-Poison,Bug-Rock,Bug-Steel,Bug-Water,Dark,Dark-Dragon,Dark-Fighting,Dark-Fire,Dark-Flying,Dark-Ghost,Dark-Ice,Dark-Psychic,Dark-Steel,Dragon,Dragon-Electric,Dragon-Fairy,Dragon-Fire,Dragon-Flying,Dragon-Ground,Dragon-Ice,Dragon-Psychic,Electric,Electric-Dragon,Electric-Fairy,Electric-Fire,Electric-Flying,Electric-Ghost,Electric-Grass,Electric-Ice,Electric-Normal,Electric-Steel,Electric-Water,Fairy,Fairy-Flying,Fighting,Fighting-Dark,Fighting-Flying,Fighting-Psychic,Fighting-Steel,Fire,Fire-Dragon,Fire-Fighting,Fire-Flying,Fire-Ground,Fire-Normal,Fire-Psychic,Fire-Rock,Fire-Steel,Fire-Water,Flying,Flying-Dragon,Ghost,Ghost-Dark,Ghost-Dragon,Ghost-Fire,Ghost-Flying,Ghost-Grass,Ghost-Poison,Grass,Grass-Dark,Grass-Dragon,Grass-Fairy,Grass-Fighting,Grass-Flying,Grass-Ground,Grass-Ice,Grass-Poison,Grass-Psychic,Grass-Steel,Ground,Ground-Dark,Ground-Dragon,Ground-Electric,Ground-Fire,Ground-Flying,Ground-Ghost,Ground-Psychic,Ground-Rock,Ground-Steel,Ice,Ice-Flying,Ice-Ghost,Ice-Ground,Ice-Psychic,Ice-Water,Normal,Normal-Fairy,Normal-Fighting,Normal-Flying,Normal-Grass,Normal-Ground,Normal-Psychic,Normal-Water,Poison,Poison-Bug,Poison-Dark,Poison-Dragon,Poison-Fighting,Poison-Flying,Poison-Ground,Poison-Water,Psychic,Psychic-Dark,Psychic-Fairy,Psychic-Fighting,Psychic-Fire,Psychic-Flying,Psychic-Ghost,Psychic-Grass,Rock,Rock-Bug,Rock-Dark,Rock-Dragon,Rock-Fairy,Rock-Fighting,Rock-Flying,Rock-Grass,Rock-Ground,Rock-Ice,Rock-Psychic,Rock-Steel,Rock-Water,Steel,Steel-Dragon,Steel-Fairy,Steel-Fighting,Steel-Flying,Steel-Ghost,Steel-Ground,Steel-Psychic,Steel-Rock,Water,Water-Dark,Water-Dragon,Water-Electric,Water-Fairy,Water-Fighting,Water-Flying,Water-Ghost,Water-Grass,Water-Ground,Water-Ice,Water-Poison,Water-Psychic,Water-Rock,Water-Steel
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,Grass-Poison,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,Grass-Poison,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,Grass-Poison,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,Grass-Poison,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,Fire,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False,Fire,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0

In [122]:
# Quiero quedarme con las columnas de los tipos combinados además de la total

cols_combo = pokemon.columns[14:].to_list()
cols_combo.insert(0,'Total')
cols_combo

['Total',
 'Bug',
 'Bug-Electric',
 'Bug-Fighting',
 'Bug-Fire',
 'Bug-Flying',
 'Bug-Ghost',
 'Bug-Grass',
 'Bug-Ground',
 'Bug-Poison',
 'Bug-Rock',
 'Bug-Steel',
 'Bug-Water',
 'Dark',
 'Dark-Dragon',
 'Dark-Fighting',
 'Dark-Fire',
 'Dark-Flying',
 'Dark-Ghost',
 'Dark-Ice',
 'Dark-Psychic',
 'Dark-Steel',
 'Dragon',
 'Dragon-Electric',
 'Dragon-Fairy',
 'Dragon-Fire',
 'Dragon-Flying',
 'Dragon-Ground',
 'Dragon-Ice',
 'Dragon-Psychic',
 'Electric',
 'Electric-Dragon',
 'Electric-Fairy',
 'Electric-Fire',
 'Electric-Flying',
 'Electric-Ghost',
 'Electric-Grass',
 'Electric-Ice',
 'Electric-Normal',
 'Electric-Steel',
 'Electric-Water',
 'Fairy',
 'Fairy-Flying',
 'Fighting',
 'Fighting-Dark',
 'Fighting-Flying',
 'Fighting-Psychic',
 'Fighting-Steel',
 'Fire',
 'Fire-Dragon',
 'Fire-Fighting',
 'Fire-Flying',
 'Fire-Ground',
 'Fire-Normal',
 'Fire-Psychic',
 'Fire-Rock',
 'Fire-Steel',
 'Fire-Water',
 'Flying',
 'Flying-Dragon',
 'Ghost',
 'Ghost-Dark',
 'Ghost-Dragon',
 'Ghost-Fi

In [123]:
# Defino la matriz de correlación a estudiar

df_corr_combo = pokemon[cols_combo].corr()

In [125]:
df_corr_combo.head()

,Total,Bug,Bug-Electric,Bug-Fighting,Bug-Fire,Bug-Flying,Bug-Ghost,Bug-Grass,Bug-Ground,Bug-Poison,Bug-Rock,Bug-Steel,Bug-Water,Dark,Dark-Dragon,Dark-Fighting,Dark-Fire,Dark-Flying,Dark-Ghost,Dark-Ice,Dark-Psychic,Dark-Steel,Dragon,Dragon-Electric,Dragon-Fairy,Dragon-Fire,Dragon-Flying,Dragon-Ground,Dragon-Ice,Dragon-Psychic,Electric,Electric-Dragon,Electric-Fairy,Electric-Fire,Electric-Flying,Electric-Ghost,Electric-Grass,Electric-Ice,Electric-Normal,Electric-Steel,Electric-Water,Fairy,Fairy-Flying,Fighting,Fighting-Dark,Fighting-Flying,Fighting-Psychic,Fighting-Steel,Fire,Fire-Dragon,Fire-Fighting,Fire-Flying,Fire-Ground,Fire-Normal,Fire-Psychic,Fire-Rock,Fire-Steel,Fire-Water,Flying,Flying-Dragon,Ghost,Ghost-Dark,Ghost-Dragon,Ghost-Fire,Ghost-Flying,Ghost-Grass,Ghost-Poison,Grass,Grass-Dark,Grass-Dragon,Grass-Fairy,Grass-Fighting,Grass-Flying,Grass-Ground,Grass-Ice,Grass-Poison,Grass-Psychic,Grass-Steel,Ground,Ground-Dark,Ground-Dragon,Ground-Electric,Ground-Fire,Ground-Flying,Ground-Ghost,Ground-Psychic,Ground-Rock,Ground-Steel,Ice,Ice-Flying,Ice-Ghost,Ice-Ground,Ice-Psychic,Ice-Water,Normal,Normal-Fairy,Normal-Fighting,Normal-Flying,Normal-Grass,Normal-Ground,Normal-Psychic,Normal-Water,Poison,Poison-Bug,Poison-Dark,Poison-Dragon,Poison-Fighting,Poison-Flying,Poison-Ground,Poison-Water,Psychic,Psychic-Dark,Psychic-Fairy,Psychic-Fighting,Psychic-Fire,Psychic-Flying,Psychic-Ghost,Psychic-Grass,Rock,Rock-Bug,Rock-Dark,Rock-Dragon,Rock-Fairy,Rock-Fighting,Rock-Flying,Rock-Grass,Rock-Ground,Rock-Ice,Rock-Psychic,Rock-Steel,Rock-Water,Steel,Steel-Dragon,Steel-Fairy,Steel-Fighting,Steel-Flying,Steel-Ghost,Steel-Ground,Steel-Psychic,Steel-Rock,Water,Water-Dark,Water-Dragon,Water-Electric,Water-Fairy,Water-Fighting,Water-Flying,Water-Ghost,Water-Grass,Water-Ground,Water-Ice,Water-Poison,Water-Psychic,Water-Rock,Water-Steel
Total,1.000000,-0.178699,-0.016537,0.047979,0.008309,-0.017369,-0.058753,-0.037054,-0.037625,-0.089742,-0.000052,0.058471,-0.049015,0.001030,0.002506,-0.007142,0.021270,0.038960,-0.002131,0.014572,-0.020922,-0.008394,-0.021410,0.072266,0.045708,0.072266,0.149777,0.057482,0.128737,0.127066,-0.033589,0.051610,-0.001211,0.025052,0.067801,0.001445,0.025052,0.025052,-0.020922,0.003359,0.025052,-0.034786,0.016660,-0.053630,0.017675,0.019150,-0.017964,0.058418,-0.036359,0.058692,0.045261,0.084398,0.003359,0.001210,0.042550,-0.007407,0.048659,0.048659,0.060506,-0.018834,-0.005164,0.014724,0.102264,-0.023934,-0.005054,-0.023651,0.011026,-0.054546,-0.001758,0.057512,-0.023010,0.045152,-0.008667,0.026528,0.019906,-0.024255,-0.005263,-0.015911,-0.042744,-0.024446,-0.002131,0.010593,0.098824,0.059068,-0.017581,-0.014658,0.010182,0.021511,0.011270,0.008309,0.013249,-0.012846,-0.023010,-0.012846,-0.064085,-0.069525,0.064682,-0.092626,-0.012570,-0.003571,0.038583,-0.007407,-0.064380,-0.031014,0.000459,0.017380,-0.016746,-0.011993,0.029188,-0.033965,0.054856,0.072266,-0.027628,0.104173,0.048659,0.010560,0.048659,0.048659,-0.022829,-0.004219,0.089736,0.002671,0.084386,0.042757,0.052859,-0.004219,-0.039954,0.002671,0.002045,0.011035,-0.012642,0.038960,0.072266,0.004212,0.042757,0.008822,0.010730,0.052154,0.041902,-0.002611,-0.050266,0.042585,0.073033,-0.016746,-0.041801,0.062210,-0.024374,-0.011526,-0.045257,-0.001128,0.039182,-0.004317,0.030361,-0.003756,0.028003
Bug,-0.178699,1.000000,-0.007377,-0.007377,-0.007377,-0.019665,-0.005213,-0.012809,-0.007377,-0.018183,-0.009040,-0.013844,-0.005213,-0.016578,-0.009040,-0.007377,-0.009040,-0.011685,-0.007377,-0.007377,-0.007377,-0.007377,-0.017398,-0.005213,-0.005213,-0.005213,-0.012809,-0.011685,-0.009040,-0.010445,-0.027538,-0.005213,-0.005213,-0.005213,-0.011685,-0.005213,-0.005213,-0.005213,-0.007377,-0.009040,-0.005213,-0.020368,-0.007377,-0.023595,-0.005213,-0.005213,-0.009040,-0.007377,-0.028062,-0.005213,-0.013844,-0.012809,-0.009040,-0.007377,-0.007377,-0.005213,-0.005213,-0.005213,-0.007377,-0.007377,-0.016578,-0.005213,-0.007377,-0.009040,-0.007377,-0.016578,-0.010445,-0.030

In [127]:
# Aplicamos misma lógica que en el apartado anterior

max_corr = df_corr_combo.iloc[0,1:].max()
id_max = df_corr_combo.iloc[0,1:].idxmax()
f'La correlación máxima es: {round(max_corr,4)}, y es para el tipo {id_max}'

'La correlación máxima es: 0.1498, y es para el tipo Dragon-Flying'

In [131]:
df_corr_combo.iloc[0,1:].sort_values(ascending=False).head(10)

Dragon-Flying       0.149777
Dragon-Ice          0.128737
Dragon-Psychic      0.127066
Psychic-Fighting    0.104173
Ghost-Dragon        0.102264
Ground-Fire         0.098824
Rock-Dark           0.089736
Fire-Flying         0.084398
Rock-Fairy          0.084386
Water-Dragon        0.073033
Name: Total, dtype: float64

## **CONCLUSIONES:**

Vemos que la hipótesis que planteamos al principio no era correcta, ya que la combinación con mayor correlación es Dragon-Flying, y Dragon-Psychic es la tercera

Vemos también que la respuesta obtenida es consistente, ya que Dragon era el tipo con mayor correlación, con bastante diferencia con Psychic.